In [25]:
import os
import glob
import json
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string
import tiktoken

# Load in the 343 Personas that mimic the study done

In [26]:
# Open the JSON file in read mode and load its contents
file_path = "new_personas_343_shortened_ind.json"
with open(file_path, "r") as json_file:
    personas = json.load(json_file)
    
#personas = personas[:3]
print("Total Personas:", len(personas))

Total Personas: 343


In [27]:
personas[75]

{'Name': 'Jeremy Harris',
 'Age': '75',
 'Ethnicity': 'White',
 'Gender': 'Male',
 'Income': '2200',
 'Education Level': "Bachelor's Degree or Higher",
 'Political Affiliation': 'Right Leaning',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Occupation': 'Retired Business Executive.',
 'Media Consumption': 'As an expert in market research, I recommend the following media outlets for Jeremy Harris, based on his demographic information:\n\n1. Fox News - aligned with his right-leaning political affiliation.\n2. The Wall Street Journal - trusted for financial news and economic analysis.\n3. The Drudge Report - known for conservative-leaning news coverage.\n4. The Washington Times - a right-leaning news source.\n\nJeremy Harris, a retired business executive, may consume 2-3 hours of media per day, given his age and geographic location.',
 'Voting History': 'As for me, I have consistently voted in the past, usually for candidates who share my ri

In [29]:
persona_role = GPT.open_file(f"Prompts/Polling/persona_gpt.txt").replace('<<PERSONA>>', str(personas[75]))
q = "Do you think transgender people should be able to change the sex on their birth certificate? \n Yes\n No"
question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', q)
print(persona_role)
print(question_in)

You will be answering political survey questions as the following persona: {'Name': 'Jeremy Harris', 'Age': '75', 'Ethnicity': 'White', 'Gender': 'Male', 'Income': '2200', 'Education Level': "Bachelor's Degree or Higher", 'Political Affiliation': 'Right Leaning', 'Geographic Location': 'Urban Montana, United States', 'Veteran Status': 'Not a Veteran', 'Occupation': 'Retired Business Executive.', 'Media Consumption': 'As an expert in market research, I recommend the following media outlets for Jeremy Harris, based on his demographic information:\n\n1. Fox News - aligned with his right-leaning political affiliation.\n2. The Wall Street Journal - trusted for financial news and economic analysis.\n3. The Drudge Report - known for conservative-leaning news coverage.\n4. The Washington Times - a right-leaning news source.\n\nJeremy Harris, a retired business executive, may consume 2-3 hours of media per day, given his age and geographic location.', 'Voting History': 'As for me, I have consis

# Load the survey questions
    - Using questions from phone interview

In [4]:
class Question:
    def __init__(self, number, question, possible_responses):
        self.number = number
        self.question = question
        self.possible_responses = possible_responses

    def format_for_gpt(self):
        formatted_question = self.question + '\n'
        formatted_responses = '\n'.join(self.possible_responses)

        return formatted_question + formatted_responses

In [5]:
questions = [
    Question(1, "Do you plan to vote in the 2022 midterm election?", ["Yes", "No", "Neither"]),
    Question(2, "At this point in time, do you approve or disapprove of President Joe Biden’s performance in office?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(3, "Do you approve or disapprove of Biden’s student loan forgiveness plan?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(4, "At this point in time, do you approve or disapprove of the performance of the US Congress?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(5, "When electing the United States president, do you think we should amend the Constitution to allow the candidate who receives the most votes to win, or do you think we should keep the current system where the candidate who wins the Electoral College vote wins?", ["Amend the Constitution", "Keep the current system"]),
    Question(6, "At this point in time, do you approve or disapprove of Governor Greg Gianforte’s job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(7, "Gianforte’s administration recently attempted to implement a rule banning transgender people from changing their designated sex status on their birth certificate. Do you think transgender people should be able to change the sex on their birth certificate?", ["Yes", "No", "Neither Yes/No"]),
    Question(8, "Are you in US House District 1 or 2?", ["District 1", "District 2", "Don't Know"]),
    Question(9, "If you had to choose today, which of the following candidates would you choose to support for House Representative in District 1?", ["Zinke (R)","Tranel (D)", "Lamb (L)", "Don't Know"]),
    Question(10, "At this point in time, do you approve or disapprove of Congressperson Matt Rosendale’s job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(11, "If you had to choose today, which of the following candidates would you choose to support for House Representative in District 2?", ["Rosendale (R)","Ronning (D)", "Rankin (L)", "Buchanan (I)", "Don't Know"]),
    Question(12, "At this point in time, do you approve or disapprove of Senator Steve Daines’ job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(13, "At this point in time, do you approve or disapprove of Senator Jon Tester’s job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(14, "Do you trust your representatives’ ability to govern efficiently and in a non-divisive manner?", ["Trust", "Distrust", "Neither Trust/Distrust"]),
    Question(15, "Do you trust your local police or sheriff’s department?", ["Trust", "Distrust", "Neither Trust/Distrust"]),
    Question(16, "Have you heard of the Indian Education for All law?", ["Yes", "No", "Neither Yes/No"]),
    Question(17, "The Indian Education for All law is based on a Montana constitutional requirement to encourage learning about American Indian culture and history. Do you believe the money from this law is being used effectively to educate the general student body within Montana’s school system?", ["Yes", "No", "Neither Yes/No"]),
    Question(18, "Do you agree or disagree that there is a mental health crisis in Montana?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(19, "Do you agree or disagree that more fun indoor activities could help the depression rate in Montana?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(20, "Do you agree or disagree that businesses in your community are struggling to find employees?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(21, "Do you agree or disagree that current Montana agricultural practices are environmentally sustainable?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(22, "Do you agree or disagree that Montana officials should work to protect public lands?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(23, "Do you agree or disagree with the Supreme Court's decision to overturn Roe v. Wade?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(24, "When it comes to abortion issues, would you consider yourself to be pro-life or pro-choice?", ["Pro-life", "Pro-choice", "Neither Pro-life/Pro-choice"]),
    Question(25, "Do you agree or disagree with the efforts to remove potentially controversial books from public schools?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(26, "Do you agree or disagree that teachers with a clean background check and training should be allowed to carry a gun in the classroom?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(27, "Do you agree or disagree with using the death penalty as punishment for convicted murder?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(28, "Do you trust mainstream media?", ["Trust", "Distrust", "Neither Trust/Distrust"])
]


In [6]:
len(questions[:13])

13

In [20]:
def create_questions_json(questions):
    question_data = []
    for question in questions:
        question_data.append({
            "number": question.number,
            "text": question.question,
            "possible_responses": question.possible_responses
        })

    json_string = json.dumps(question_data)
    return json_string
json_string = create_questions_json(questions[:7])
print(json_string)

[{"number": 1, "text": "Do you plan to vote in the 2022 midterm election?", "possible_responses": ["Yes", "No", "Neither"]}, {"number": 2, "text": "At this point in time, do you approve or disapprove of President Joe Biden\u2019s performance in office?", "possible_responses": ["Approve", "Disapprove", "Neither Approve/Disapprove"]}, {"number": 3, "text": "Do you approve or disapprove of Biden\u2019s student loan forgiveness plan?", "possible_responses": ["Approve", "Disapprove", "Neither Approve/Disapprove"]}, {"number": 4, "text": "At this point in time, do you approve or disapprove of the performance of the US Congress?", "possible_responses": ["Approve", "Disapprove", "Neither Approve/Disapprove"]}, {"number": 5, "text": "When electing the United States president, do you think we should amend the Constitution to allow the candidate who receives the most votes to win, or do you think we should keep the current system where the candidate who wins the Electoral College vote wins?", "po

# Initialize Survey Responses
    - A survey response will each have
        - Persona responding
        - Dictionary of responses mapped to the question number
            - Doing dictionary instead of list just in case of a failure of gpt to answer a question
            - The indices will have correct values

In [15]:
import threading
class SurveyResponse:
    def __init__(self, person):
        self.person = person
        self.survey_responses = {}
        self.lock = threading.Lock()  # Create a lock for synchronization


    def ask_question(self, question):
        global total_prompt_tokens, total_completion_tokens
        # Load the Persona role and question
        print("got here, qn", question.number)
        persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(self.person))
        question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question.format_for_gpt())
        
        #print(persona_role, question_in)
        try:
            resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.25, tokens = 300)
            print("GPT ran")
        except:
            resp = ""
            usage = 0
    
        total_completion_tokens += usage['completion_tokens']
        total_prompt_tokens += usage['prompt_tokens']
    
        with self.lock:  # Acquire the lock before modifying the dictionary
            print(resp)
            self.survey_responses[question.number] = resp
        print("sv",self.survey_responses[question.number])
        
    def __str__(self):
        response_str = f"Survey Response for: {self.person}\n"
        response_str += "Survey Responses:\n"
        for question_number, response in self.survey_responses.items():
            response_str += f"Question {question_number}: {response}\n"
        return response_str

In [16]:
SURVEY_RESPONSES = []
for p in personas:
    SURVEY_RESPONSES.append(SurveyResponse(p))
print("Total Surveys:", len(SURVEY_RESPONSES))

Total Surveys: 343


# Conduct Survey to Personas

## Saving Function
    - After each question, save data to be safe

In [17]:
class SurveyResponseEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, SurveyResponse):
            return {
                "person": obj.person,
                "survey_responses": obj.survey_responses
            }
        return super().default(obj)

In [3]:
qc = 0

total_prompt_tokens = 0
total_completion_tokens = 0

# def ask_question_threaded(SURVEY_RESPONSE, question):
#     SURVEY_RESPONSE.ask_question(question)
    
def ask_question_threaded(SURVEY_RESPONSE, question):  
    global total_prompt_tokens, total_completion_tokens
    # Load the Persona role and question
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(SURVEY_RESPONSE.person))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question.format_for_gpt())

    #print(persona_role, question_in)
    try:
        resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.25, tokens = 250)
    except:
        resp = ""
        usage = 0

    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']

    SURVEY_RESPONSE.survey_responses[str(question.number)] = resp

for question in questions:
#     if qc == 1:
#         break
#     qc += 1
    # Create a thread pool executor with 10 workers
    print(f"Starting Question: {str(question.number)}\n")
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Submit tasks to the executor for each persona
        futures = [executor.submit(ask_question_threaded, SURVEY_RESPONSE, question) for SURVEY_RESPONSE in SURVEY_RESPONSES]

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)
    
        # Save responses as JSON
    with open("survey_responses_343_gpt_4.json", "w") as file:
        json.dump(SURVEY_RESPONSES, file, cls=SurveyResponseEncoder, indent=4)

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

NameError: name 'questions' is not defined

In [24]:
responses = []

total_prompt_tokens = 0
total_completion_tokens = 0
def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_gpt.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/multi_question.txt").replace('<<QUESTION>>', json_string)
    
    try:
        resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.6, tokens = 2048)
    except:
        return "",0
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses = list(executor.map(thread_func, range(50,51)))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
prompt_cost = (total_prompt_tokens)/1000*0.03
completion_cost = (total_completion_tokens)/1000*0.06
print("Prompt Cost = $", str(prompt_cost))
print("Completion Cost = $", str(completion_cost))
print("Total Cost = $", str(prompt_cost + completion_cost))

50 1. {"Reasoning": "As a conservative and a believer in individual responsibility, I believe that voting is an essential civic duty. I have voted consistently in the past and plan to vote in the 2022 midterm election. I prioritize researching the candidates' platforms and values to ensure they align with my own before casting my vote.", "Answer": "Yes"}

2. {"Reasoning": "As a conservative and a supporter of smaller government, lower taxes, and individual freedom, I am generally not in favor of President Joe Biden's policies and actions. I believe that his policies will lead to increased government control and higher taxes. Therefore, I disapprove of his performance in office.", "Answer": "Disapprove"}

3. {"Reasoning": "As a conservative, I am generally not in favor of government programs that provide financial assistance to individuals. While I sympathize with those who struggle with student loan debt, I do not believe that the government should use taxpayer money to forgive their l

# Analyze Results

In [44]:
resps_out = {}
for i in range(len(questions)):
    resps_out[i+1] = []

for i in range(len(questions)):
    for s in SURVEY_RESPONSES:
        
        try:
            resp_data = json.loads(s.survey_responses[str(i+1)])
            resp_val = resp_data['Answer']
            resps_out[i+1].append(resp_val)   
        except Exception as e:
            print("Invalid Format", str(e))
            continue

In [50]:
def count_responses(responses):
    # Count occurrences of each unique value
    counts = {}
    total_responses = len(responses)

    for response in responses:
        if response not in counts:
            counts[response] = 0
        counts[response] += 1

    # Print counts and percentages
    print("Counts of each response:")
    for response, count in counts.items():
        percentage = (count / total_responses) * 100
        print(f"{response}: {count} ({percentage:.2f}%)")

In [51]:
for i, resp in resps_out.items():
    count_responses(resp)
    print()

Counts of each response:
Yes: 3 (100.00%)

Counts of each response:
Disapprove: 1 (33.33%)
Approve: 2 (66.67%)

Counts of each response:
Disapprove: 2 (66.67%)
Approve: 1 (33.33%)

Counts of each response:
Neither Approve/Disapprove: 2 (66.67%)
Disapprove: 1 (33.33%)

Counts of each response:
Keep the current system: 3 (100.00%)

Counts of each response:
Approve: 3 (100.00%)

Counts of each response:
Yes: 3 (100.00%)

Counts of each response:
District 1: 2 (66.67%)
Don't Know: 1 (33.33%)

Counts of each response:
Zinke (R): 2 (66.67%)
Don't Know: 1 (33.33%)

Counts of each response:
Approve: 3 (100.00%)

Counts of each response:
Rosendale (R): 2 (66.67%)
I would need to research and understand the candidates and their platforms before making a decision on who to support. At this time, I do not have enough information to choose a candidate.: 1 (33.33%)

Counts of each response:
Approve: 3 (100.00%)

Counts of each response:
Approve: 2 (66.67%)
Neither Approve/Disapprove: 1 (33.33%)

Cou

# Process Responses

In [94]:
def person_to_string(person):
    persona_string = []
    for key, value in person.items():
        # Some keys require specific phrasing to form a directive sentence
        if key == 'Name':
            persona_string.append(f"Your name is {value}.")
        elif key == 'Age':
            persona_string.append(f"You are {value} years old.")
        elif key == 'Ethnicity':
            persona_string.append(f"You identify as {value}.")
        elif key == 'Gender':
            persona_string.append(f"You are a {value} individual.")
        elif key == 'Income':
            persona_string.append(f"You make an annual income of around {value} USD.")
        elif key == 'Education Level':
            persona_string.append(f"Your highest level of education is {value}.")
        elif key == 'Political Affiliation':
            persona_string.append(f"You lean towards the {value} politically.")
        elif key == 'Geographic Location':
            persona_string.append(f"You currently live in {value}.")
        elif key == 'Veteran Status':
            persona_string.append(f"You are {value}.")
        elif key == 'Occupation':
            persona_string.append(f"You work as a {value}.")
        else:
            # The remaining keys don't require specific phrasing, so we can just add their values
            # These are written in first person so we can use them directly
            persona_string.append(value)
    
    # Join all elements of persona_string into a single string, separated by space
    return " ".join(persona_string)